<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

#   [Apprentissage en Grande Dimension](https://github.com/wikistat/High-Dimensional-Learning ): avec  <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>:  Détection d'anomalies dans des données fonctionnelles 

**Résumé** Tutoriel de détection d'anomalies dans des données fonctionnelles. Les données simulées reproduisent les comportement de télémétries observées lors de tests réalisés sur des sattellites avant lancement. Ce sujet est issu du travail de thèse de Clémentine Barreyre chez Airbus Defense and Space. 


** Librairies **
La liste  des *packages* ou librairies que nous utiliserons : la librairie **`wavethress`** ou la librairie **`waveslim`** pour calculer les coefficients d'ondelettes et la librairie **`e1071`** sera utilisée pour la méthode  one-class SVM, et la librairie **`DMwR`** pour la méthode Local Outlier Factor.   


In [ ]:
library(wavethresh)
library(waveslim)

library("e1071")
library(DMwR)

## 1  Description et lecture des données

On dispose de 480 jours de télémesures. Chaque jour comporte 256 mesures  à intervalles de temps réguliers. Le jeu de données comporte le vecteur des temps et le vecteur des valeurs de la télémesure aux différents instants.

Nous sommes dans un contexte d'apprentissage semi-supervisé : nous savons que les 240 derniers jours ne comportent pas d'anomalies. Les 240 premiers jours comportent quelques anomalies que nous allons chercher à détecter. 



### 1.1 Lecture des données  et statistiques élémentaires


In [ ]:
TM=read.csv("../data/DonneesSimulees/TM_Saved_simulated_2017-06-23.dat", row.names=1)
summary(TM)
dim(TM)



Dans un premier temps, on se place donc dans un cadre non supervisé.  On n'utilise  pas l'information fournie par les  240 premières courbes qui ont été labellisées comme étant exemptes de toute anomalie.
On utilise seulement les 240 premières courbes, qui contiennent un petit nombre d'anomalies. 


In [ ]:
p = 256

TM_480 = matrix(TM[,2], nrow = p)
dim(TM_480)

TM_240 = TM_480[,c(1:240)]

dim(TM_240)

On précise quelles sont les 8 courbes atypiques. Ceci est inconnu du statisticien et servira seulement pour la validation de nos résultats. 
On introduit des couleurs différentes selon le type d'anomalies :

- 4 anomalies de "forme " représentées en rouge : changement de forme ou d'amplitude des courbes 
- 3 anomalies locales représentées en rose : ajout d'un pic ou de bruit ou d'une valeur atypique
- 1 anomalie de périodicité (deux périodes au lieu d'une) représentée en orange.


In [ ]:
anomalies = rep(0,240)
anomalies[c(134,201, 156, 26, 220, 6, 70, 98)] = 1
anomalies0 = rep(0,240)

color = rep(1, ncol(TM_240))
color[c(26,220,6,70)] = "red"
color[c(134,201,156)] = "pink3"
color[98] = "orange"                            



### 1.2 Tracés des courbes
#### Ensembe des courbes

In [ ]:
 plot(TM_240[,1],col=color[1],type='l',ylim=c(-2,4))
for (j in 2:240) {
    lines(TM_240[,j],col=color[j],type='l')}

#### Quelques courbes 

In [ ]:
matplot(TM_240[,c(134:156)],type="l",col=color[c(134:156)],lty=1)

#### Ensemble des courbes atypiques

In [ ]:
matplot(TM_240[,c(134,201, 156, 26, 220, 6, 70, 98)],type="l",col=color[c(134,201, 156, 26, 220, 6, 70, 98)],lty=1)

## 2. Analyse en composantes principales
### 2.1 Sur le jeu de données contenant des anomalies

In [ ]:
acp=prcomp(t(TM_240), scale=TRUE)
plot(acp)
biplot(acp)

### 2.2 Sur le jeu de données ne contenant pas d'anomalies: 

In [ ]:
acp2=prcomp(t(TM_480[,c(241:480)]), scale=TRUE)
plot(acp2)
biplot(acp2)

## 3 Définition des *features* 

Pour détecter les anomalies, on peut utiliser les données brutes, mais ce n'est pas toujours le plus efficace. On peut aussi essayer d'extraire de nouvelles variables, appelées caractéristiques (*features*), qui permettront peut-être de mettre davantage en lumière les anomalies. Pour cela, on peut par exemple utiliser les coefficients de la projection des données brutes sur des bases orthonormées: base de composantes principales, ou bases d'ondelettes. 


### 3.1 Coefficients dans la base des composantes principales


In [ ]:
# Calcul des Features obtenues à l'aide de l'ACP : 

ACP = prcomp(t(TM_240))
phi= ACP$rotation
Theta = t(TM_480)%*%phi

index = min(which(cumsum(ACP$sdev)/sum(ACP$sdev)>0.95))
index

In [ ]:

# Definition des features utilisées pour détecter les anomalies : 

# Les coefficients pour les courbes ne contenant pas d'anomalies
set00 = Theta[241:480,1:index]

# Pour le jeu de données contenant des anomalies 
# données brutes
set0= t(TM_240)

# Coefficients sur la base des composantes principales
set1 = Theta[1:240,]

# Seulement les coefficients associés aux plus grandes variances
set2 = Theta[1:240,1:index]

### 3.2 Coefficients dans une base d'ondelettes

In [ ]:
Train=t(TM_240)
nlevels =8 # 2**8=256

TrainT = NULL # table de coefficients d'ondelettes
threshTrainT = NULL # table de coefficients d'ondelettes seuillés

for (i in 1:nrow(Train)){ 
    xTrain = Train[i,]
    wtTrain = dwt(xTrain, n.levels=8)
    threshwtTrain=universal.thresh(wtTrain, max.level = 4, hard = TRUE)
    TrainT = rbind(TrainT,unlist(wtTrain))
    threshTrainT =rbind(threshTrainT ,unlist(threshwtTrain))
    }

TrainT=data.frame(TrainT)
threshTrainT=data.frame(threshTrainT)

dim(TrainT)
dim(threshTrainT)

# Nombre de coefficients non nuls : 

sum(sum(TrainT!=0))
sum(sum(threshTrainT!=0))

In [ ]:
# Definition des features utilisées pour détecter les anomalies : 

# Tous les coefficients sur la base d'ondelettes :
set3= TrainT

# Seulement les coefficients d'ondelettes de niveau le plus fin J=7 
set4= TrainT[,1:128]

# Les coefficients des niveaux J=0 à  6
set5=TrainT[, 128:256]

# Les coefficients seuillés : 
set6=threshTrainT

### 3.3 Analyse en composantes principales sur les coefficients d'ondelettes

In [ ]:
acp=prcomp(t(set4))
plot(acp)
biplot(acp)

In [ ]:
acp=prcomp(t(set5))
plot(acp)
biplot(acp)

In [ ]:
acp=prcomp(t(set6))
plot(acp)
biplot(acp)

## 4 Détection d'anomalies par différentes méthodes
### 4.1 Classification ascendante hierarchique 
#### Données sans anomalies

In [ ]:
H = hclust(d = dist(set00), method = "single")
plot(H)

In [ ]:
c = cutree(H, k = 5)
table(c, anomalies0)

In [ ]:
plot(H$height)

#### CAH sur données brutes

In [ ]:
H0 = hclust(d = dist(set0), method = "single")
plot(H0)

In [ ]:
c0 = cutree(H0, k = 5)
table(c0, anomalies)

In [ ]:
H0$order

In [ ]:
plot(H0$height)

#### CAH sur composantes principales

In [ ]:
H1= hclust(d = dist(set1), method = "single")
plot(H1)

In [ ]:
c1 = cutree(H1, k = 5)
table(c1, anomalies)

In [ ]:
H1$order

In [ ]:
plot(H1$height)

#### CAH sur les premières composantes principales

In [ ]:
H2 = hclust(d = dist(set2), method = "single")
plot(H2)

In [ ]:
c2 = cutree(H2, k = 5)
table(c2, anomalies)

In [ ]:
H2$order
plot(H2$height)

#### CAH sur les  coefficients d'ondelettes

In [ ]:
H3 = hclust(d = dist(set3), method = "single")
plot(H3)

In [ ]:
c3 = cutree(H3, k = 5)
table(c3, anomalies)

In [ ]:
H3$order

In [ ]:
plot(H3$height)

#### CAH sur  les  coefficients d'ondelettes de niveau le plus fin

In [ ]:
H4 = hclust(d = dist(set4), method = "single")
plot(H4)

In [ ]:
c4 = cutree(H4, k = 5)
table(c4, anomalies)

In [ ]:
H4$order

In [ ]:
plot(H4$height)

#### Fausse alarme

In [ ]:
plot(TM_240[,25],type="l",col=color[25],lty=1)

#### CAH sur les  coefficients d'ondelettes de niveaux 0 à 6 

In [ ]:
H5 = hclust(d = dist(set5), method = "single")
plot(H5)

In [ ]:
c5 = cutree(H5, k = 5)
table(c5, anomalies)

In [ ]:
H5$order

In [ ]:
plot(H5$height)

#### CAH sur les  coefficients d'ondelettes  seuillés 

In [ ]:
H6 = hclust(d = dist(set6), method = "single")
plot(H6)

In [ ]:
c6 = cutree(H6, k = 5)
table(c6, anomalies)

In [ ]:
H6$order

In [ ]:
plot(H6$height)

** Q ** Comparer les résultats. Quelles jeux de features donnent des résultats comparables ? Lequel donne des résultats différents. Ceci est-il cohérent avec les types d'anomalies considérées ? 

### 4.2 One Class SVM
#### Avec les coefficients dans la base des composantes principales

In [ ]:
ocsvm.acp = svm(set2, nu=0.05)
resu.svm.acp= ocsvm.acp$fitted
table(resu.svm.acp, anomalies)

In [ ]:
which(resu.svm.acp=="FALSE")

** Q. ** Commenter les résultats et tracer les courbes présentant des anomalies qui sont détectées et celles qui ne le sont pas, ainsi que les fausses alarmes. 

#### Avec les coefficients d'ondelettes de niveau le plus fin

In [ ]:
ocsvm.ond7 = svm(set4, nu=0.05)
resu.svm.ond7= ocsvm.ond7$fitted
table(resu.svm.ond7, anomalies)

In [ ]:
which(resu.svm.ond7=="FALSE")

#### Avec les coefficients d'ondelettes de niveau 0 à 6

In [ ]:
ocsvm.ond = svm(set5, nu=0.05)
resu.svm.ond= ocsvm.ond$fitted
table(resu.svm.ond, anomalies)

In [ ]:
which(resu.svm.ond=="FALSE")

** Q. ** Commenter les résultats obtenus avec les coefficients d'ondelettes et retrouvez quelles sont les courbes atypiques qui sont détectées ainsi que les "faux positifs". 

### 4.3 Local Outlier Factor
#### Avec les coefficients dans la base des composantes principales

In [ ]:
lof.acp1 = lofactor(data = set1, k = 5)
resu.lof.acp1= (lof.acp1>2)
table(resu.lof.acp1, anomalies)

In [ ]:
lof.acp2 = lofactor(data = set2, k = 5)
resu.lof.acp2= (lof.acp2>2)
table(resu.lof.acp2, anomalies)

#### Avec les données brutes 

In [ ]:
lof.acp0 = lofactor(data = set0, k = 5)
resu.lof.acp0= (lof.acp0>2)
table(resu.lof.acp0, anomalies)

** Q. ** Commenter ces résultats et retrouvez quelles sont les courbes atypiques qui sont détectées ainsi que les "faux positifs"

#### Avec les coefficients d'ondelettes

In [ ]:
# Tous les coefficients
lof.ond = lofactor(data = set3, k =5)
resu.lof.ond= (lof.ond>2)
table(resu.lof.ond, anomalies)

In [ ]:
# Niveau le plus fin
lof.ond7 = lofactor(data = set4, k = 5)
resu.lof.ond7= (lof.ond7>2)
table(resu.lof.ond7, anomalies)

In [ ]:
# Niveau 0 à 6 
lof.ond06 = lofactor(data = set5, k = 5)
resu.lof.ond06= (lof.ond06>2)
table(resu.lof.ond06, anomalies)

In [ ]:
# Coefficients seuillés
lof.ondthresh = lofactor(data = set6, k = 5)
resu.lof.ondthresh= (lof.ondthresh>2)
table(resu.lof.ondthresh, anomalies)

## 5 Visualisation des résultats

## 5.1 Une fonction pour visualiser les résultats

In [ ]:
plot.LOF= function(Vec, main ="Log(Local Outlier Factor)", lim = c(4), ymax = max(Vec)){
  plot(Vec, xlab = "Days", ylab = "log(Local Outlier Factor)", main = main, ylim = c(0, ymax), 
       pch = 1+ 1*(Vec > min(lim)))
  anom = which(color!="1")
  for(i in anom){
    abline(v = i, col = color[i], lty = 2)
  }
  points (anom, Vec[anom], col = 2 + 5*(Vec[anom]>min(lim)) - 4*(Vec[anom]>=max(lim)) ,cex = 2, pch = 16)
  points (anom, Vec[anom], col = 1 ,cex = 2, pch = 1)
  for(i in lim){
    abline(h=i, lwd = 2)
  }
  legend("topright", c("LOF", "Anom Detected","Anom not detected", "Pattern anom", "Periodicity anom", "Local Anom", "Limit"), lty = c(NA,NA,NA,2,2,2,1),pch = c(1,16,16,NA,NA,NA,NA), 
         lwd = c(1,1,1,1,1,1,2), col = c(1,3,2,2,"orange", "pink3", 1))
}

** Q. ** Commenter les résultats obtenus avec les coefficients d'ondelettes et retrouvez quelles sont les courbes atypiques qui sont détectées ainsi que les "faux positifs". Essayez de déterminer comment optimiser le choix du paramètre **k**. 

## 5.2 Résultats obtenus par  *One-Class-SVM* 

In [ ]:
# OCSVM on wavelet coef : 
plot.LOF(1 - resu.svm.ond, ymax = 2, lim = 0.5, main ="One-Class SVM on wavelets coef")

In [ ]:
plot.LOF(1 - resu.svm.ond7, ymax = 2, lim = 0.5, main ="One-Class SVM on wavelets coef (level 7)")

In [ ]:
# Ce qui fonctionne le mieux : OCSVM on first ACP coef 
plot.LOF(1 - resu.svm.acp, ymax = 2, lim = 0.5, main ="One-Class SVM on first acp coef")

### 5.3 Résultats obtenus par  *Local Outlier Factor*: 

In [ ]:
#LOF on  ACP coef
plot.LOF(lof.acp, ymax = 20, lim = 2, main = "LOF on ACP coef")

In [ ]:
plot.LOF(lof.acp2, ymax = 20, lim = 2, main = "LOF on first ACP coef")

In [ ]:
#LOF on wavelet coef
plot.LOF(lof.ond, ymax = 20, lim = 2, main = " LOF on wavelets coef")

In [ ]:
plot.LOF(lof.ond7, ymax = 20, lim = 2, main = "LOF on wavelets coef (max level)")

**Q** Ajouter d'autres méthodes de détection d'anomalies